In [1]:
%matplotlib inline
import phate
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Embedding DLA fractal tree

In [2]:
#generate DLA tree
M, C = phate.tree.gen_dla(n_dim = 100, n_branch = 20, branch_length = 100,n_drop = 0, rand_multiplier = 2, seed=37, sigma = 4)

#instantiate phate_operator
phate_operator = phate.PHATE(n_components=2, a=10, k=5, t=30, mds='classic', knn_dist='euclidean', mds_dist='euclidean')

#run phate with classic MDS
Y_cmds = phate_operator.fit_transform(M)

#run phate with metric MDS
phate_operator.reset_mds(mds="metric")
Y_mmds = phate_operator.fit_transform(M)

#run phate with nonmetric MDS
phate_operator.reset_mds(mds="nonmetric")
Y_nmmds = phate_operator.fit_transform(M)

Bulding kNN graph and diffusion operator...
Built graph and diffusion operator in 8.96 seconds
Calculating diffusion potential...
Calculated diffusion potential in 50.62 seconds
Embedding data using classic MDS...
Embedded data in 25.85 seconds
Finished PHATE embedding in 25.85 seconds
Using precomputed diffusion operator
Using precomputed diffusion potential
Embedding data using metric MDS...
Embedded data in 31.17 seconds
Finished PHATE embedding in 31.17 seconds
Using precomputed diffusion operator
Using precomputed diffusion potential
Embedding data using nonmetric MDS...


Process ForkPoolWorker-48:
Process ForkPoolWorker-37:
Process ForkPoolWorker-47:
Process ForkPoolWorker-36:
Process ForkPoolWorker-34:
Process ForkPoolWorker-41:
Process ForkPoolWorker-46:
Process ForkPoolWorker-40:
Process ForkPoolWorker-39:
Process ForkPoolWorker-38:
Process ForkPoolWorker-35:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multip

KeyboardInterrupt: 

In [ ]:
#plot
f, (ax1, ax2, ax3) = plt.subplots(3)

ax1.scatter(Y_cmds[:,0], Y_cmds[:, 1], s=10, c=C)
ax1.set_xlabel("phate 1")
ax1.set_ylabel("phate 2")
ax1.set_title("PHATE embedding of DLA fractal tree - Classic MDS")

ax2.scatter(Y_mmds[:,0], Y_mmds[:, 1], s=10, c=C)
ax2.set_xlabel("phate 1")
ax2.set_ylabel("phate 2")
ax2.set_title("PHATE embedding of DLA fractal tree - Metric MDS")

ax3.scatter(Y_nmmds[:,0], Y_nmmds[:, 1], s=10, c=C)
ax3.set_xlabel("phate 1")
ax3.set_ylabel("phate 2")
ax3.set_title("PHATE embedding of DLA fractal tree - Nonmetric MDS")

### Embedding mESC

In [7]:
phate_operator.fit_transform(M, diffOp=phate_operator.diffOp_)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

array([[  5.11990853e-02,   5.14902780e-03,   1.48917990e-02, ...,
          1.46130092e-24,   1.65038329e-30,   5.75066806e-42],
       [  6.90929955e-03,   6.87022543e-02,   1.44237890e-02, ...,
          2.39064642e-24,   1.14413506e-27,   9.14775334e-27],
       [  1.39694698e-02,   1.00833150e-02,   4.80280507e-02, ...,
          1.19782906e-21,   4.27276015e-21,   3.11840157e-23],
       ..., 
       [  3.39295030e-24,   4.13660834e-24,   2.96483090e-21, ...,
          1.18877604e-01,   3.58162639e-02,   4.00623948e-02],
       [  8.33844506e-30,   4.30793126e-27,   2.30131469e-20, ...,
          7.79368223e-02,   2.58679764e-01,   1.58826405e-01],
       [  1.60684771e-41,   1.90485568e-26,   9.28871634e-23, ...,
          4.82120372e-02,   8.78372779e-02,   1.43060132e-01]])